<h1>Genetic Algorithms</h1> <br/>
<strong>In this computer assignment we are going to use Genetic algorithms and improving them.
We are going to use crossover and mutation algorithms on our generation and create a new generation which should be better than our last generation of chromosomes.<br/>
For this assignment we need to define some concepts.</strong> <br/> <br/>
<h2>Chromosome</h2> <strong>
هر کروموزوم یک جواب احتمالی برای ما هست که مجموعه ای از ژن هاست
    </strong>
<h2>Gene</h2> <strong>
هر ژن یک شیفت است که در آن ایندکس دکتر هایی که کار میکنند را مینویسیم
</strong><h2>Fitness</h2>
<strong>
برای این قسمت به اینطور عمل میکنیم که هر پزشک باید هر 3 تا شرط را ارضا کند (یکی این که 3 شب پشت هم کار نکند، یکی این که اگر شب کار کند صبح و عصر فردایش کار نکند و آخری اینکه از ظرفیت خود بیشتر نشود ) و اگر همه پزشک ها هر 3 شرط را ارضا کنند به جواب نهایی میرسیم ( امتیاز هر جمعیت جمع این امتیاز هاست )
و همینطور برای هر شیفت چک میکنیم که آیا تعداد پزشک ها کمتر از ماکسیمیم و بیشتر از مینیمم هست و به ازای هر شیفت اگر این شیفت برقرار باشد به آن یک امتیاز میدهیم
</strong>
<h2>Probability</h2>
<strong>
Shows the possibility of each chromosome. We show it with 'p'. <br/>
$P_i$ = $fitness_i$ / $\sum_j^n fitness_j$
</strong>

In [4]:
import random
import time

testFile = "test1.txt"

def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    file.close()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    return [days, list(range(doctors)), maxCapacity, allShifts]

class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        self.popSize = 300
        self.pc = 0.65
        self.pm = 0.4
        self.goalReach = False
        
        self.chromosomes = self.generateInitialPopulation()

        self.crossOverPoints = 3 * self.days - 1
        self.elitismPercentage = 0.16
        
        
    def generateInitialPopulation(self):
        chromosomes = []
        for i in range(self.popSize):
            tmp = []
            for day in range(len(self.allShifts)):
                for shift in range(len(self.allShifts[day])):
                    generateRandom = random.randint(0,self.doctors - 1)
                    tmpLst = list(map(str,random.sample(range(0,self.doctors - 1),generateRandom)))
                    chromosome = ''.join(tmpLst)
                    tmp.append(chromosome)
            chromosomes.append(tmp)
        return chromosomes
        
    def crossOver(self, chromosome1, chromosome2):
        newChromosome1 = chromosome1[:]
        newChromosome2 = chromosome2[:]
        for i in range(self.crossOverPoints):
            generateRandom = random.random()
            if generateRandom >= self.pc:
                newChromosome1[i] = chromosome2[i][:]
                newChromosome2[i] = chromosome1[i][:]
        return newChromosome1, newChromosome2
        
    def mutate(self, chromosome):
        for i in range(self.crossOverPoints):
            generateRandom = random.random()
            if generateRandom < self.pm:
                # length = random.randint(0,self.doctors - 1)
                newNumber = list(map(str,random.sample(range(0,self.doctors - 1),len(chromosome[i]))))                   
                chromosome[i] = "".join(newNumber)
        return chromosome
    
    def lessThanCapacity(self,chromosome):
        doctors = {}
        lessThanMaxCapacity = 0
        for i in range(self.doctors):
            doctors[i] = 0
        for i in range(len(chromosome)):
            for j in range(len(chromosome[i])):
                doctors[int(chromosome[i][j])] += 1
        for i in doctors:
            if doctors[i] <= self.maxCapacity: lessThanMaxCapacity += 1
        return lessThanMaxCapacity
    
    def workNightCantWorkTomorrow(self,chromosome):
        validDocs = {}
        workNightCantWorkTomorrow = 0
        for i in range(self.doctors):
            validDocs[i] = True
        for i in range(2,len(chromosome),3):
            doctors = []
            for j in range(len(chromosome[i])):
                doctors.append(int(chromosome[i][j]))
            tmp = min(3,len(chromosome) - i - 1)
            for next in range(1,tmp):
                for iter in range(len(chromosome[i + next])):
                    if int(chromosome[i + next][iter]) in doctors:
                        validDocs[int(chromosome[i][j])] = False
        for i in validDocs:
            if validDocs[i]: workNightCantWorkTomorrow += 1
        return workNightCantWorkTomorrow

    def threeNightsInARowCantWork(self,chromosome):
        validDocs = {}   
        threeNightsInARowCantWork = 0
        for i in range(self.doctors):
            validDocs[i] = True
        for i in range(2,len(chromosome),3):
            doctors = []
            for j in range(len(chromosome[i])):
                doctors.append(chromosome[i][j])
            if i + 6 < len(chromosome):
                for j in range(len(doctors)):
                    if doctors[j] in chromosome[i + 3] and doctors[j] in chromosome[i + 6]:
                        validDocs[int(doctors[j])] = False
        for i in validDocs:
            if validDocs[i]: threeNightsInARowCantWork += 1
        return threeNightsInARowCantWork
    
    def checkProperNumInShift(self,chromosome):
        tmp = 0
        for i in range(len(chromosome)):
            min = self.allShifts[i // 3][i % 3][0]
            max = self.allShifts[i // 3][i % 3][1]
            if len(chromosome[i]) <= max and len(chromosome[i]) >= min:
                tmp += 1
        return tmp
        
    def calculateFitness(self, chromosome):
        lessThanMaxCapacity = self.lessThanCapacity(chromosome)
        workNightCantWorkTomorrow = self.workNightCantWorkTomorrow(chromosome)
        threeNightsInARowCantWork = self.threeNightsInARowCantWork(chromosome)
        checkProperNumInShift = self.checkProperNumInShift(chromosome)

        if lessThanMaxCapacity + threeNightsInARowCantWork + workNightCantWorkTomorrow == self.doctors * 3 and checkProperNumInShift == (3 * self.days): 
            self.goalReach = True
            self.result = chromosome
        return lessThanMaxCapacity + workNightCantWorkTomorrow + threeNightsInARowCantWork + checkProperNumInShift
    
    
    def generateNewPopulation(self):
        for num in range(self.popSize):
            chooseTwoPpl = random.sample(range(0,self.popSize - 1),2)
            i , j = chooseTwoPpl[0] , chooseTwoPpl[1]
            self.chromosomes[i] , self.chromosomes[j] = self.crossOver(self.chromosomes[i],self.chromosomes[j])
            i = random.randint(0,self.popSize - 1)
            self.chromosomes[i] = self.mutate(self.chromosomes[i])
        return
    
    def schedule(self):
        while not self.goalReach:
            fitness = []
            for i in range(self.popSize):
                fitness.append(self.calculateFitness(self.chromosomes[i]))
            self.chromosomes = [x for _,x in sorted(zip(fitness,self.chromosomes),reverse=True)]
            firstOnes = self.chromosomes[:round(self.elitismPercentage * self.popSize)][:]
            self.generateNewPopulation()
            self.chromosomes[:round(self.elitismPercentage * self.popSize)] = firstOnes[:]
        self.showResult()
        return

    def showResult(self):
        result = ""
        for i in range(len(self.result)):
            if len(self.result[i]) == 0: result += "empty "
            for j in range(len(self.result[i])):
                result += self.result[i][j] + ","
            if i % 3 == 2:
                result = result[:-1] + "\n"
            else:
                result = result[:-1] + " "
        fileName = "output" + testFile[4] + ".txt"
        file = open(fileName,"w")
        file.write(result)
        file.close()

if __name__ == "__main__":

    fileInfo = readInput(testFile)

    start = time.time()
    scheduler = JobScheduler(fileInfo)
    scheduler.schedule()
    end = time.time()

    print("test {}: ".format(testFile[4]), '%.2f'%(end - start), 'sec')

test 1:  8.92 sec


<strong>تست 1 در حدود 5 ثانیه و تست 2 در حدود 30 ثانیه اجرا می شود </strong>